In [ ]:
%matplotlib notebook
import h5py
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import fftconvolve
from scipy.optimize import leastsq, curve_fit
from scipy.integrate import simps, cumtrapz
from glob import glob

In [ ]:
def _fitfunc(t, f0, tau):
    return f0*np.exp(-t/tau)

def _linfitfunc(t, p):
    return p[0] + p[1]*t

def fitfunc(p, t):
    return p[0] + p[1]*t
def errfunc(p, t, y):
    return fitfunc(p, t) - y


def get_block_data(group, name, dim=3):
    t_data = []
    data = []
    block = group[name]['value'][:]
    count = group[name]['count'][:]
    block /= count.reshape((-1, 1, 1, 1))
    dt = group[name]['time'][()]
    for i in range(block.shape[0]):
        t = dt*np.arange(block.shape[1])*block.shape[1]**i
        t_data.append(t[1:])
        data.append(block[i,1:,:,:].reshape((-1,dim)))

    return np.concatenate(t_data), np.concatenate(data)


In [ ]:
runs = glob('/pierre/work/20161212/cceq_profile_*/*.h5')
runs = glob('/home/share/nanomotors/cceq/20161212/cceq_profile_*/*.h5')

runs.sort()

msd_all = []
vacf_all = []
tvacf_all = []
pvacf_all = []
wacf_all = []

for f in runs:
    a = h5py.File(f, 'r')
    group = a['block_correlators']

    msd_t, msd_data = get_block_data(group, 'mean_square_displacement')
    msd_all.append(msd_data)

    vacf_t, vacf_data = get_block_data(group, 'velocity_autocorrelation')
    vacf_all.append(vacf_data)

    do_pvacf = 'parallel_velocity_autocorrelation' in group
    if do_pvacf:
        pvacf_t, pvacf_data = get_block_data(group, 'parallel_velocity_autocorrelation')
        pvacf_all.append(pvacf_data)

    do_tvacf = 'transverse_velocity_autocorrelation' in group
    if do_tvacf:
        tvacf_t, tvacf_data = get_block_data(group, 'transverse_velocity_autocorrelation')
        tvacf_all.append(tvacf_data)

    do_wacf = 'planar_angular_velocity_autocorrelation' in group
    if do_wacf:
        wacf_t, w_data = get_block_data(group, 'planar_angular_velocity_autocorrelation', dim=1)
        wacf_all.append(w_data.flatten())
        
    a.close()

msd_all = np.array(msd_all)
vacf_all = np.array(vacf_all)
pvacf_all = np.array(pvacf_all)
tvacf_all = np.array(tvacf_all)
wacf_all = np.array(wacf_all)

In [ ]:
msd_all.shape

In [ ]:
plt.ylabel(r'$\langle (\mathbf{r}(\tau) - \mathbf{r}(0))^2 \rangle$')

m = msd_all.mean(axis=0)
plt.plot(msd_t, m, marker='o')
m = m[...,:2].sum(axis=-1)

mask = msd_t>100
solution, ierr = leastsq(errfunc, [0, 0.1], args=(msd_t[mask], m[mask]))
intercept, D = solution

# MSD = 2 d D t = 4 D t -> The coefficient of the linear fit must be divided by 4
D = D/4
plt.plot(msd_t, fitfunc((intercept, 2*D), msd_t))
print("D_parallel + D_perp = ", 2*D)
plt.xlabel(r'$\tau$')
plt.loglog()


In [ ]:
plt.ylabel(r'tvacf')
m = tvacf_all.sum(axis=-1).mean(axis=0)
m = tvacf_all[...,:2].sum(axis=-1).mean(axis=0)
m = cumtrapz(m, tvacf_t, initial=0)
plt.plot(1/tvacf_t, m)
#plt.xscale('log')
plt.ylim(1e-4, 0.008)

mask = (1/tvacf_t>0.005) * (1/tvacf_t<0.1)
solution, ierr = leastsq(errfunc, [0.1, -0.1], args=(1/tvacf_t[mask], m[mask]))
inv_t = np.linspace(1e-4, 1, 100)
plt.plot(inv_t, fitfunc(solution, inv_t))
plt.xlim(0, 0.05)

print("Number of points:", np.sum(mask))
print(solution)

plt.ylabel(r'pvacf')
m = pvacf_all.sum(axis=-1).mean(axis=0)
m = pvacf_all[...,:2].sum(axis=-1).mean(axis=0)
m = cumtrapz(m, pvacf_t, initial=0)
plt.plot(1/pvacf_t, m)
#plt.xscale('log')

mask = (1/pvacf_t>=0.005)*(1/pvacf_t<=0.1)
solution, ierr = leastsq(errfunc, [0.1, -0.1], args=(1/pvacf_t[mask], m[mask]))
inv_t = np.linspace(1e-4, 1, 100)
plt.plot(inv_t, fitfunc(solution, inv_t))
plt.xlim(0, 0.05)
plt.ylim(0, 0.003)

print("Number of points:", np.sum(mask))
print(solution)

In [ ]:
plt.ylabel(r'pvacf')
m = pvacf_all.sum(axis=-1).mean(axis=0)
m = pvacf_all[...,:2].sum(axis=-1).mean(axis=0)
m = cumtrapz(m, pvacf_t, initial=0)
plt.plot(1/pvacf_t, m)
#plt.xscale('log')

mask = (1/pvacf_t>=0.005)*(1/pvacf_t<=0.05)
solution, ierr = leastsq(errfunc, [0.1, -0.1], args=(1/pvacf_t[mask], m[mask]))
inv_t = np.linspace(1e-4, 1, 100)
plt.plot(inv_t, fitfunc(solution, inv_t))
plt.xlim(0, 0.05)
plt.ylim(0, 0.003)

print("Number of points:", np.sum(mask))
print(solution)

In [ ]:
m[mask]

In [ ]:
plt.figure()
ax1 = plt.subplot(211)
plt.ylabel(r'tvacf')
m = tvacf_all[...,:2].sum(axis=-1).mean(axis=0)
plt.plot(tvacf_t, m, marker='o')
plt.plot(tvacf_t, cumtrapz(m, tvacf_t, initial=0))
plt.xscale('log')
plt.ylim(-0.001,0.0025)

ax2 = plt.subplot(212, sharex=ax1)
plt.ylabel(r'pvacf')
m = pvacf_all[...,:2].sum(axis=-1).mean(axis=0)
plt.plot(pvacf_t, m, marker='o')
plt.plot(pvacf_t, cumtrapz(m, pvacf_t, initial=0))
plt.xscale('log')
plt.ylim(-0.001,0.0025)

In [ ]:
m = tvacf_all[...,:2].sum(axis=-1).mean(axis=0)
s = tvacf_all[...,:2].sum(axis=-1).std(axis=0)
plt.plot(tvacf_t, m, 'k-')
plt.plot(tvacf_t, m-s, 'k--')
plt.plot(tvacf_t, m+s, 'k--')

m = pvacf_all[...,:2].sum(axis=-1).mean(axis=0)
s = pvacf_all[...,:2].sum(axis=-1).std(axis=0)

plt.plot(pvacf_t, m, 'k-')
plt.plot(pvacf_t, m-s, 'k--')
plt.plot(pvacf_t, m+s, 'k--')

plt.xscale('log')

In [ ]:
m = wacf_all.mean(axis=0)
s = wacf_all.std(axis=0)
plt.plot(wacf_t, m, marker='o')
plt.plot(wacf_t, cumtrapz(m, wacf_t, initial=0))

popt, pcov = curve_fit(_fitfunc, wacf_t, m, sigma=s)
monomer_mass = 1130
I = 2*monomer_mass*(6.7/2)**2
T=1/3

plt.plot(wacf_t, _fitfunc(wacf_t, *popt))
plt.ylabel(r'wacf')
plt.loglog()
plt.ylim(1e-9, 1e-3)
gamma = I/popt[1]
print('wacf exponential fit: D=%5.3e' % (T/gamma))


In [ ]:
wacf_all.shape